# ZRP Example Usage
The purpose of this notebook is to illustrate how to use ZRP, the main class of the zrp package that processes user input data &  returns race/ethnicity predictions

In [20]:
print('asdf')

asdf


In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [3]:
from os.path import join, expanduser
import pandas as pd
import sys
import os
import re
import warnings

## Set source code path here

In [4]:
warnings.filterwarnings(action='once')
home = expanduser('~')

src_path = os.getcwd()
sys.path.append(src_path)

In [5]:
from zrp import ZRP
from zrp.prepare.utils import load_file

## Load sample data for prediction
Load list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [25]:
nj_mayors = load_file(src_path + "/2022-nj-mayors.csv")
nj_mayors.shape

(565, 18)

In [26]:
nj_mayors

,MUNI CODE,MUNI NAME,COUNTY,ADDRESS 1,ADDRESS 2,CITY,STATE,ZIP,PHONE,FAX,MAYOR NAME,TERM START,TERM END,FORM,TERM LEGNTH,EMAIL,SOCIAL MEDIA HANDLE,Municipal Contact List
0,1330,Aberdeen Township,Monmouth,One Aberdeen Square,NaN,Aberdeen,NJ,07747-2300,(732) 583-4200,NaN,Fred Tagliarini,NaN,12/31/2025,COUNCIL-MANAGER,4,fred.tagliarini@aberdeennj.org,NaN,NaN
1,0101,Absecon City,Atlantic,Absecon Municipal Complex,500 Mill Road,Absecon,NJ,08201,(609) 641-0663,(609) 645-5098,Kimberly Horton,NaN,12/31/2024,MAYOR-COUNCIL,3,khorton@abseconnj.org,NaN,NaN
2,1001,Alexandria Township,Hunterdon,782 Frenchtown Road,NaN,Milford,NJ,08848,(908) 996-7071,NaN,Gabe Plumer,NaN,12/31/2022,TOWNSHIP,3,clerk@alexandrianj.gov,NaN,NaN
3,2101,Allamuchy Township,Warren,Post Office Box A,NaN,Allamuchy,NJ,07820,(908) 852-5132,NaN,Rosemary Tuohy,NaN,12/31/2024,FAULKNER ACT,3,mayor@allamuchynj.org,NaN,NaN
4,0201,Allendale Borough,Bergen,500 West Crescent Avenue,NaN,Allendale,NJ,07401,(201) 818-4400,NaN,Ari Bernstein,NaN,12/31/2022,NaN,NaN,aribernstein@allendalenj.gov,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0269,Wood-Ridge Borough,Bergen,85 Humboldt Street,NaN,Wood-Ridge,NJ,07075-2344,(201) 939-0202,NaN,Paul A Sarlo,NaN,12/31/2023,NaN,NaN,psarlo@njwoodridge.org,NaN,NaN
561,1715,Woodstown Borough,Salem,Post Office Box 286,NaN,Woodstown,NJ,08098,(856) 769-2200,NaN,Donald Dietrich,NaN,12/31/2023,NaN,NaN,Don.dietrich@comcast.net,NaN,NaN
562,0824,Woolwich Township,Gloucester,120 Village Green Drive,NaN,Woolwich Township,NJ,08085-3180,(856) 467-2666,NaN,Craig Frederick,NaN,12/31/2024,NaN,NaN,cfrederick@woolwichtwp.org,NaN,NaN
563,0340,Wrightstown Borough,Burlington,21 Saylors Pond Road,NaN,Wrightstown,NJ,08562,(609) 723-4450,(609) 723-7137,Donald Cottrell,NaN,12/31/2022,NaN,NaN,mayor@wrightstownborough.com,NaN,NaN


### Wrangle NJ mayor data for predictions
Prepare the NJ mayor data.  This parsing of the NJ mayors file will leave some NA's, but it is sufficient for demonstration purposes


In [27]:
zrp_sample = pd.DataFrame(columns=['first_name', 'middle_name', 'last_name', 'house_number', 'street_address', 'city', 'state', 'zip_code'])

Prepare Names

In [28]:
split_mayor_names = nj_mayors['MAYOR NAME'].str.split(' ')
zrp_sample['first_name'] = split_mayor_names.str[0]
zrp_sample['last_name'] = split_mayor_names.str[-1]

City, State, Zip

In [29]:
zrp_sample['city'] = nj_mayors['CITY']
zrp_sample['state'] = nj_mayors['STATE']
zrp_sample['zip_code'] = nj_mayors['ZIP']

Address

In [30]:
zrp_sample['house_number'] = nj_mayors['ADDRESS 1'].str.extract('([0-9]+)')
zrp_sample['street_address'] = nj_mayors['ADDRESS 1'].str.extract('.*[0-9]+([^0-9]+)')


In [31]:
zrp_sample['ZEST_KEY'] = zrp_sample.index.astype(str)  #must specify key to establish correspondence between inputs and outputs
zrp_sample

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Fred,NaN,Tagliarini,NaN,NaN,Aberdeen,NJ,07747-2300,0
1,Kimberly,NaN,Horton,NaN,NaN,Absecon,NJ,08201,1
2,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
3,Rosemary,NaN,Tuohy,NaN,NaN,Allamuchy,NJ,07820,3
4,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
...,...,...,...,...,...,...,...,...,...
560,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
561,Donald,NaN,Dietrich,286,NaN,Woodstown,NJ,08098,561
562,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562
563,Donald,NaN,Cottrell,21,Saylors Pond Road,Wrightstown,NJ,08562,563


### Invoke the Zest Race Predictor on the sample data

In [32]:
%%time
zest_race_predictor = ZRP()
zest_race_predictor.fit()
zrp_output = zest_race_predictor.transform(zrp_sample)

  0%|          | 0/565 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.0s
100%|██████████| 565/565 [00:00<00:00, 16405.55it/s]

Data is loaded
   [Start] Validating input data
     Number of observations: 565
     Is key unique: True
       (Warning!!) middle_name is 100.0% missing, this may impact the ability to return race approximations
       (Warning!!) house_number is 11.327433628318584% missing, this may impact the ability to return race approximations
       (Warning!!) street_address is 18.230088495575224% missing, this may impact the ability to return race approximations
Directory already exists
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace

[Start] Preparing geo data

  The following states are included in the data: ['NJ']
   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
      ...address cleaning



[Parallel(n_jobs=-1)]: Done 565 out of 565 | elapsed:    0.0s finished


      ...replicating address
         ...Base
         ...Map street suffixes...
         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=1003)
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=1003)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping


100%|██████████| 1/1 [00:05<00:00,  5.79s/it]

   [Completed] Validating input geo data
Directory already exists
Output saved
   [Completed] Mapping geo data

[Completed] Preparing geo data

[Start] Preparing ACS data
   [Start] Validating ACS input data
     Number of observations: 565
     Is key unique: True
       (Warning!!) middle_name is 100.0% missing, this may impact the ability to return race approximations
       (Warning!!) street_address is 18.230088495575224% missing, this may impact the ability to return race approximations



   [Completed] Validating ACS input data

   ...loading ACS lookup tables
   ... combining ACS & user input data
 ...Copy dataframes
 ...Block group
 ...Census tract
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete
[Complete] Preparing ACS data

   [Start] Validating pipeline input data
     Number of observations: 1624
     Is key unique: False
       (Warning!!) middle_name is 100.0% missing, this may impact the ability to return race approximations
       (Warning!!) street_address is 12.31527093596059% missing, this may impact the ability to return race approximations
   [Completed] Validating pipeline input data



100%|██████████| 1/1 [00:00<00:00, 930.00it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1/1 [00:00<00:00, 744.86it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1/1 [00:00<00:00, 578.13it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


Directory already exists
Output saved
Output saved
CPU times: user 52.2 s, sys: 8.04 s, total: 1min
Wall time: 53.9 s


### Inspect the output and join

In [33]:
zrp_output

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_block_group,source_census_tract,source_zip_code,source_bisg
0,Fred,NaN,Tagliarini,NaN,NaN,Aberdeen,NJ,07747-2300,0,0.000445,0.000367,0.001623,0.000926,0.996639,WHITE,0.0,0.0,1.0,0.0
1,Kimberly,NaN,Horton,NaN,NaN,Absecon,NJ,08201,1,0.020005,0.022037,0.385148,0.041842,0.530967,WHITE,0.0,0.0,1.0,0.0
2,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2,0.000124,0.000097,0.033672,0.002466,0.963641,WHITE,1.0,0.0,0.0,0.0
3,Rosemary,NaN,Tuohy,NaN,NaN,Allamuchy,NJ,07820,3,0.000584,0.000502,0.001105,0.002336,0.995472,WHITE,0.0,0.0,1.0,0.0
4,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4,0.016728,0.008392,0.002305,0.050236,0.922338,WHITE,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560,0.000146,0.000116,0.000424,0.000582,0.998732,WHITE,1.0,0.0,0.0,0.0
561,Donald,NaN,Dietrich,286,NaN,Woodstown,NJ,08098,561,0.009935,0.000523,0.007039,0.008381,0.974121,WHITE,0.0,0.0,1.0,0.0
562,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562,0.012623,0.018644,0.190711,0.023586,0.754436,WHITE,0.0,0.0,1.0,0.0
563,Donald,NaN,Cottrell,21,Saylors Pond Road,Wrightstown,NJ,08562,563,0.011897,0.027253,0.118932,0.023193,0.818726,WHITE,1.0,0.0,0.0,0.0


### Check the most likely Hispanic 

In [34]:
zrp_output.nlargest(10, "HISPANIC")

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_block_group,source_census_tract,source_zip_code,source_bisg
377,Hector,NaN,Lora,330,Passaic Street,Passaic,NJ,07055-5815,377,0.000121,0.000230,0.001202,0.982070,0.016378,HISPANIC,1.0,0.0,0.0,0.0
286,Marcial,NaN,Mojena,249,NaN,Columbus,NJ,08022,286,0.001020,0.000309,0.001682,0.965945,0.031043,HISPANIC,0.0,0.0,1.0,0.0
536,Gabriel,NaN,Rodriguez,428,th Street,West New York,NJ,07093-2222,536,0.001721,0.005429,0.002667,0.958142,0.032041,HISPANIC,0.0,0.0,1.0,0.0
388,Helmin,NaN,Caba,NaN,NaN,Perth Amboy,NJ,08861,388,0.021552,0.000426,0.008470,0.953645,0.015907,HISPANIC,0.0,0.0,1.0,0.0
236,Alberto,NaN,Santos,402,Kearny Avenue,Kearny,NJ,07032,236,0.019993,0.000571,0.003235,0.951424,0.024777,HISPANIC,1.0,0.0,0.0,0.0
543,Ray,NaN,Arroyo,101,Washington Avenue,Westwood,NJ,07675,543,0.004734,0.006675,0.004117,0.940691,0.043783,HISPANIC,1.0,0.0,0.0,0.0
499,Manuel,NaN,Figueiredo,NaN,NaN,Union,NJ,07083-3597,499,0.000594,0.000619,0.001788,0.916256,0.080742,HISPANIC,0.0,0.0,1.0,0.0
378,Andre,NaN,Sayegh,125,st Floor,Paterson,NJ,07505-1414,378,0.075077,0.000379,0.005269,0.889816,0.029458,HISPANIC,0.0,0.0,1.0,0.0
418,Ramopn,NaN,Hache,131,North Maple Avenue,Ridgewood,NJ,07450-3236,418,0.001468,0.000549,0.001547,0.846190,0.150247,HISPANIC,0.0,0.0,1.0,0.0
556,Carlos,NaN,Rendo,188,Pascack Road,Woodcliff Lake,NJ,07677-7921,556,0.000315,0.000395,0.001300,0.837821,0.160169,HISPANIC,0.0,0.0,1.0,0.0


### Check the most likely Black

In [35]:
zrp_output.nlargest(10, "BLACK")

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_block_group,source_census_tract,source_zip_code,source_bisg
215,Dahlia,NaN,Vertreese,NaN,NaN,Hillside,NJ,07205,215,0.000495,0.000783,0.973164,0.009093,0.016466,BLACK,0.0,0.0,1.0,0.0
229,Anthony,NaN,Vauss,NaN,NaN,Irvington,NJ,07111-2412,229,0.002764,0.001271,0.970029,0.016721,0.009215,BLACK,0.0,0.0,1.0,0.0
549,Tiffani,NaN,Worthy,1,Salem Road,Willingboro,NJ,08046,549,0.000840,0.000422,0.963874,0.008897,0.025967,BLACK,1.0,0.0,0.0,0.0
343,Ras,NaN,Baraka,920,Broad Street,Newark,NJ,07102,343,0.000688,0.000089,0.961769,0.003171,0.034282,BLACK,1.0,0.0,0.0,0.0
370,Dwayne,NaN,Warren,29,North Day Street,Orange,NJ,07050,370,0.002516,0.012330,0.960430,0.005826,0.018898,BLACK,0.0,0.0,1.0,0.0
258,Derek,NaN,Armstead,301,North Wood Avenue,Linden,NJ,07036-4296,258,0.016190,0.009768,0.941253,0.002946,0.029844,BLACK,0.0,0.0,1.0,0.0
397,Adrian,NaN,Mapp,515,Watchung Avenue,Plainfield,NJ,07060-1720,397,0.004066,0.010136,0.939272,0.022945,0.023582,BLACK,1.0,0.0,0.0,0.0
250,Mary,NaN,Wardlow,4,East Douglas Avenue,Lawnside,NJ,08045-1597,250,0.000452,0.000842,0.925571,0.019787,0.053347,BLACK,0.0,0.0,1.0,0.0
78,Jamila,NaN,Odom-Bremmer,201,Grant Avenue,Chesilhurst,NJ,08089,78,0.013521,0.016294,0.919407,0.018952,0.031827,BLACK,1.0,0.0,0.0,0.0
433,Donald,NaN,Shaw,210,Chestnut Street,Roselle,NJ,07203-1218,433,0.004979,0.011493,0.849289,0.016776,0.117463,BLACK,1.0,0.0,0.0,0.0


BISG proxies are saved by default when `ZRP` is ran. Below we import the BISG proxies in.

In [36]:
bisg_output = pd.read_feather("artifacts/bisg_proxy_output.feather")

In [37]:
bisg_output.head()

,ZEST_KEY,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_bisg
0,56,0.038829,0.000429,0.000807,0.006858,0.937739,WHITE,1
1,202,0.010862,0.000296,0.001673,0.239643,0.742021,WHITE,1
2,204,0.009050,0.000132,0.000330,0.028292,0.956646,WHITE,1
3,224,0.001166,0.014795,0.085358,0.042606,0.836503,WHITE,1
4,248,0.007348,0.000625,0.029071,0.082111,0.861174,WHITE,1


How many proxies does BISG return?

In [38]:
f"Out of {bisg_output.shape[0]} records only {bisg_output[bisg_output.race_proxy.notna()].shape[0]} proxies are returned"  

'Out of 565 records only 438 proxies are returned'

How many proxies does ZRP return?

In [39]:
f"Out of {zrp_output.shape[0]} records {zrp_output[zrp_output.race_proxy.notna()].shape[0]} proxies are returned"  


'Out of 565 records 551 proxies are returned'

### AIAN analysis

In [154]:
import os
file = '/home/pio/shared/zrp/shared_data/raw/geo/Tribal_Block_Group_National_Shapefile/tl_2019_us_tbg.shp'
data = []

       
dictList = []
if indx == "address":
    with fiona.open(file) as src:
        for i in range(10000000):
            dictList.append(next(src)["properties"])
else:
    with fiona.open(file) as src:
        for i in range(len(src)):
            dictList.append(next(src)["properties"])
tbg = pd.DataFrame(
    dictList,
    columns=dictList[0].keys(),
)


In [153]:
import os
file = '/home/pio/shared/zrp/shared_data/raw/geo/Tribal_Census_Tract_National_Shapefile/tl_2019_us_ttract.shp'

data = []

       
dictList = []
if indx == "address":
    with fiona.open(file) as src:
        for i in range(10000000):
            dictList.append(next(src)["properties"])
else:
    with fiona.open(file) as src:
        for i in range(len(src)):
            dictList.append(next(src)["properties"])
ttract = pd.DataFrame(
    dictList,
    columns=dictList[0].keys(),
)


In [181]:
for i in df.columns:
    if i in tbg.columns:
        print(i)

AIANNHCE
TTRACTCE
TBLKGPCE
INTPTLAT
INTPTLON


In [182]:
df_short = df[df['TTRACTCE'].notna()][['AIANNHCE','TTRACTCE', 'TBLKGPCE']]

In [184]:
df_short

,AIANNHCE,TTRACTCE,TBLKGPCE
185,2865,T00100,A
161,2865,T00100,A
163,2865,T00100,A
164,2865,T00100,A
166,2865,T00100,A
...,...,...,...
11726,3355,T00100,A
11727,3355,T00100,A
11730,1140,T00300,A
11731,3355,T00100,A


In [194]:
df_short.merge(tbg ,on = ['AIANNHCE','TTRACTCE', 'TBLKGPCE'], how = 'left')

,AIANNHCE,TTRACTCE,TBLKGPCE,GEOID,NAMELSAD,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON
0,2865,T00100,A,2865T00100A,Tribal Block Group A,G2410,1607384,7844,+31.0978919,-087.5589413
1,2865,T00100,A,2865T00100A,Tribal Block Group A,G2410,1607384,7844,+31.0978919,-087.5589413
2,2865,T00100,A,2865T00100A,Tribal Block Group A,G2410,1607384,7844,+31.0978919,-087.5589413
3,2865,T00100,A,2865T00100A,Tribal Block Group A,G2410,1607384,7844,+31.0978919,-087.5589413
4,2865,T00100,A,2865T00100A,Tribal Block Group A,G2410,1607384,7844,+31.0978919,-087.5589413
...,...,...,...,...,...,...,...,...,...,...
64947,3355,T00100,A,3355T00100A,Tribal Block Group A,G2410,2421725383,23840640,+33.4885424,-110.4235097
64948,3355,T00100,A,3355T00100A,Tribal Block Group A,G2410,2421725383,23840640,+33.4885424,-110.4235097
64949,1140,T00300,A,1140T00300A,Tribal Block Group A,G2410,1284376008,122858,+34.0361942,-110.3832007
64950,3355,T00100,A,3355T00100A,Tribal Block Group A,G2410,2421725383,23840640,+33.4885424,-110.4235097


In [192]:
ttract[ttract['TTRACTCE'] == 'T00100'].sort_values('AIANNHCE')

,AIANNHCE,TTRACTCE,GEOID,NAME,NAMELSAD,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON
348,0010,T00100,0010T00100,T001,Tribal Census Tract T001,G2400,1542445497,426646,+34.7743031,-107.6602097
155,0020,T00100,0020T00100,T001,Tribal Census Tract T001,G2400,2591046,40009,+33.8054921,-116.5231199
336,0050,T00100,0050T00100,T001,Tribal Census Tract T001,G2400,36744843,83761,+30.7079432,-094.6481293
205,0080,T00100,0080T00100,T001,Tribal Census Tract T001,G2400,66224268,17539812,+42.0972144,-078.8274396
337,0095,T00100,0095T00100,T001,Tribal Census Tract T001,G2400,100606,0,+41.4776544,-120.5244037
...,...,...,...,...,...,...,...,...,...,...
280,4770,T00100,4770T00100,T001,Tribal Census Tract T001,G2400,494924189,115707,+35.5615623,-106.7442429
398,4785,T00100,4785T00100,T001,Tribal Census Tract T001,G2400,981580407,1219917,+34.9482243,-108.9364943
281,4910,T00100,4910T00100,T001,Tribal Census Tract T001,G2400,1060454,0,+39.7010141,-095.7209027
282,4930,T00100,4930T00100,T001,Tribal Census Tract T001,G2400,2785130,0,+35.3627291,-106.4945407


In [193]:
tbg[tbg['TTRACTCE'] == 'T00100'].sort_values('AIANNHCE')

,AIANNHCE,TTRACTCE,TBLKGPCE,GEOID,NAMELSAD,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON
874,0010,T00100,A,0010T00100A,Tribal Block Group A,G2410,1466775526,116317,+34.7743031,-107.6602097
47,0010,T00100,C,0010T00100C,Tribal Block Group C,G2410,25445667,310329,+35.0649752,-107.6082756
46,0010,T00100,B,0010T00100B,Tribal Block Group B,G2410,50224304,0,+35.0141687,-107.6206462
209,0020,T00100,B,0020T00100B,Tribal Block Group B,G2410,1136197,31112,+33.8121132,-116.5213363
811,0020,T00100,A,0020T00100A,Tribal Block Group A,G2410,1454849,8897,+33.8060328,-116.5189527
...,...,...,...,...,...,...,...,...,...,...
383,4785,T00100,A,4785T00100A,Tribal Block Group A,G2410,726827346,1219917,+34.8812870,-108.9606761
386,4785,T00100,B,4785T00100B,Tribal Block Group B,G2410,254753061,0,+35.1108004,-108.9513134
767,4910,T00100,A,4910T00100A,Tribal Block Group A,G2410,1060454,0,+39.7010141,-095.7209027
768,4930,T00100,A,4930T00100A,Tribal Block Group A,G2410,2785130,0,+35.3627291,-106.4945407


In [76]:
# Import raw geo files
import os
import fiona
folder = f'/home/pio/shared/zrp/shared_data/raw/geo/2019/'
j = 0
data = []

for file_name in os.listdir(folder):
    if ('faces.shp' in file_name) & ('.xml' not in file_name) :
        file = folder + file_name
        
        dictList = []

        with fiona.open(file) as src:
            for i in range(len(src)):
                dictList.append(next(src)["properties"])
        face = pd.DataFrame(
            dictList,
            columns=dictList[0].keys(),
        )
        
        check = (face[['TTRACTCE', 'TBLKGPCE']].notna().sum(axis= 1) > 0).sum() #take only files with native data
        
        if check > 0:
            data.append(face)
        j += 1
        if j == 100:
            break

faces = pd.concat(data)

ModuleNotFoundError: No module named 'fiona'

In [344]:
faces

,TFID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,BLOCKCE10,SUFFIX1CE,ZCTA5CE10,UACE10,PUMACE10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,COUSUBFP,SUBMCDFP,ESTATEFP,CONCTYFP,PLACEFP,AIANNHFP,AIANNHCE,COMPTYP,TRSUBFP,TRSUBCE,ANRCFP,TTRACTCE,TBLKGPCE,ELSDLEA,SCSDLEA,UNSDLEA,CD116FP,SLDUST,SLDLST,CSAFP,CBSAFP,METDIVFP,CNECTAFP,NECTAFP,NCTADVFP,LWFLAG,OFFSET,ATOTAL,INTPTLAT,INTPTLON,GEOID_CT,GEOID_BG,GEOID,TGEOID
0,210466333,01,051,030500,3,3015,None,36092,None,02100,01,051,030500,3,90945,None,None,None,None,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,P,N,2465,+32.6062880,-86.0853271,01051030500,010510305003,010510305003,NaN
1,210466334,01,051,030100,3,3007,None,36092,None,02100,01,051,030100,3,90855,None,None,None,35752,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,513233,+32.6322136,-86.3225168,01051030100,010510301003,010510301003,NaN
2,210466335,01,051,030100,3,3018,None,36022,None,02100,01,051,030100,3,90855,None,None,None,35752,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,332371,+32.6249528,-86.3329101,01051030100,010510301003,010510301003,NaN
3,210466336,01,051,030100,4,4013,None,36022,None,02100,01,051,030100,4,90855,None,None,None,35752,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,1942606,+32.6246830,-86.3520709,01051030100,010510301004,010510301004,NaN
4,210466337,01,051,030100,1,1010,None,36022,None,02100,01,051,030100,1,90855,None,None,None,35752,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,340781,+32.6558977,-86.3391341,01051030100,010510301001,010510301001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15099,220490331,04,007,000600,2,2102,None,85554,None,00800,04,007,000600,2,93470,None,None,None,85330,None,None,None,None,None,None,None,None,09540,None,None,04,006,006,429,37740,None,None,None,None,L,N,4794,+34.1462412,-110.9306329,04007000600,040070006002,040070006002,NaN
15100,220483007,04,007,000600,2,2107,None,85554,None,00800,04,007,000600,2,93470,None,None,None,85330,None,None,None,None,None,None,None,None,09540,None,None,04,006,006,429,37740,None,None,None,None,L,N,202598,+34.1156168,-110.9666729,04007000600,040070006002,040070006002,NaN
15101,220489108,04,007,000700,1,1045,None,85554,None,00800,04,007,000700,1,93470,None,None,None,None,None,None,None,None,None,None,None,None,09540,None,None,04,006,006,429,37740,None,None,None,None,L,N,49660,+33.8980580,-111.0008626,04007000700,040070007001,040070007001,NaN
15102,220480653,04,007,000302,1,1065,None,85541,None,00800,04,007,000302,1,92570,None,None,None,None,None,None,None,None,None,None,None,None,05760,None,None,04,006,006,429,37740,None,None,None,None,L,N,57599,+33.9656211,-111.3943270,04007000302,040070003021,040070003021,NaN


In [272]:
from zrp.prepare.preprocessing import *

In [277]:
gps = ProcessGLookUp()

data_path = '/home/pio/zrp/zrp/data/processed/state_mapping.json'
state_mapping = load_json(data_path)

faces = gps.transform(faces, state_mapping)

   [Start] Processing lookup data
     ...processing
   [Completed] Processing lookup data
     Number of observations: 159330
     Is key unique: False
{'is_empty': False, 'is_all_missing': False, 'n_obs': 159330, 'is_unique_key': False, 'pct_na': {'TFID': 0.0, 'STATEFP10': 0.0, 'COUNTYFP10': 0.0, 'TRACTCE10': 0.0, 'BLKGRPCE10': 0.0, 'BLOCKCE10': 0.0, 'SUFFIX1CE': 0.9568505617272328, 'ZCTA5CE10': 0.08632398167325676, 'UACE10': 0.9046130672189795, 'PUMACE10': 0.0, 'STATEFP': 0.0, 'COUNTYFP': 0.0, 'TRACTCE': 0.0, 'BLKGRPCE': 0.0, 'COUSUBFP': 0.0, 'SUBMCDFP': 1.0, 'ESTATEFP': 1.0, 'CONCTYFP': 1.0, 'PLACEFP': 0.7906608924872905, 'AIANNHFP': 0.5904914328751647, 'AIANNHCE': 0.5904914328751647, 'COMPTYP': 0.5904914328751647, 'TRSUBFP': 0.6446055356806628, 'TRSUBCE': 0.6446055356806628, 'ANRCFP': 0.9847109772170966, 'TTRACTCE': 0.5923429360446871, 'TBLKGPCE': 0.5923429360446871, 'ELSDLEA': 0.9275089437017511, 'SCSDLEA': 1.0, 'UNSDLEA': 0.07249105629824892, 'CD116FP': 0.0, 'SLDUST': 0.0, 'SLDL

In [294]:
faces["GEOID_CT"] = faces["STATEFP"] + faces["COUNTYFP"] + faces["TRACTCE"]
faces["GEOID_BG"] = faces["GEOID_CT"] + faces["BLKGRPCE"]
faces["GEOID"] = None
faces["GEOID"] = faces["GEOID"].fillna(faces['GEOID_BG']).fillna(faces['GEOID_CT'])
faces["TGEOID"] = faces['AIANNHCE'] + faces['TTRACTCE'] + faces['TBLKGPCE']

faces[['GEOID','GEOID_CT', 'GEOID_BG','STATEFP','COUNTYFP','TRACTCE','BLKGRPCE', 'TGEOID', 'AIANNHCE','TTRACTCE', 'TBLKGPCE']][faces['TGEOID'].notna()]#.fillna(faces[['STATEFP']])
#[['AIANNHCE','TTRACTCE', 'TBLKGPCE']]

,GEOID,GEOID_CT,GEOID_BG,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,TGEOID,AIANNHCE,TTRACTCE,TBLKGPCE
185,010510308001,01051030800,010510308001,01,051,030800,1,2865T00100A,2865,T00100,A
161,010539704003,01053970400,010539704003,01,053,970400,3,2865T00100A,2865,T00100,A
163,010539704003,01053970400,010539704003,01,053,970400,3,2865T00100A,2865,T00100,A
164,010539704003,01053970400,010539704003,01,053,970400,3,2865T00100A,2865,T00100,A
166,010539705001,01053970500,010539705001,01,053,970500,1,2865T00100A,2865,T00100,A
...,...,...,...,...,...,...,...,...,...,...,...
11726,040079404001,04007940400,040079404001,04,007,940400,1,3355T00100A,3355,T00100,A
11727,040079404001,04007940400,040079404001,04,007,940400,1,3355T00100A,3355,T00100,A
11730,040079402001,04007940200,040079402001,04,007,940200,1,1140T00300A,1140,T00300,A
11731,040079404001,04007940400,040079404001,04,007,940400,1,3355T00100A,3355,T00100,A


In [284]:
path = '/home/pio/shared/zrp/shared_data/raw/acs/2019/5yr/2019_5yr_Summary_FileTemplates/5_year_Mini_Geo.xlsx'

sheets_dict = pd.read_excel(path, sheet_name=None)
 
mini_geo = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    mini_geo = mini_geo.append(sheet)
 
mini_geo.reset_index(inplace=True, drop=True)
mini_geo['GEOID'] = mini_geo['Geography ID'].apply(lambda x: x.split('US')[1])

In [325]:
mini_geo

,State,Logical Record Number,Geography ID,Geography Name,sheet,GEOID
0,US,1,01000US,United States,us,
1,US,2,01001US,United States -- Urban,us,
2,US,3,01043US,United States -- Rural,us,
3,US,4,01089US,United States -- American Indian Reservation a...,us,
4,US,5,01090US,United States -- American Indian Reservation a...,us,
...,...,...,...,...,...,...
579674,PR,7588,86000US00982,ZCTA5 00982,pr,00982
579675,PR,7589,86000US00983,ZCTA5 00983,pr,00983
579676,PR,7590,86000US00985,ZCTA5 00985,pr,00985
579677,PR,7591,86000US00987,ZCTA5 00987,pr,00987


In [31]:
pd.set_option('display.max_columns', None)

In [312]:
faces1 = faces.merge(mini_geo[['GEOID', 'Logical Record Number']] , on = 'GEOID')
faces1.rename(columns = {'Logical Record Number':'GEOID_short'}, inplace = True)
faces1[faces1['GEOID'] == '010510308001']#[["TFID", "COUNTYFP", "STATEFP", "OFFSET"]]

,TFID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,BLOCKCE10,SUFFIX1CE,ZCTA5CE10,UACE10,PUMACE10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,COUSUBFP,SUBMCDFP,ESTATEFP,CONCTYFP,PLACEFP,AIANNHFP,AIANNHCE,COMPTYP,TRSUBFP,TRSUBCE,ANRCFP,TTRACTCE,TBLKGPCE,ELSDLEA,SCSDLEA,UNSDLEA,CD116FP,SLDUST,SLDLST,CSAFP,CBSAFP,METDIVFP,CNECTAFP,NECTAFP,NCTADVFP,LWFLAG,OFFSET,ATOTAL,INTPTLAT,INTPTLON,GEOID_CT,GEOID_BG,GEOID,TGEOID,GEOID_short
4023,210466591,01,051,030800,1,1058,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,L,N,9686,+32.5212881,-86.2099081,01051030800,010510308001,010510308001,NaN,3788
4024,210466624,01,051,030800,1,1037,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,61430,2865,R,None,None,None,T00100,A,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,142774,+32.5284632,-86.2091482,01051030800,010510308001,010510308001,2865T00100A,3788
4025,210467004,01,051,030800,1,1034,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,141939,+32.5331880,-86.2075346,01051030800,010510308001,010510308001,NaN,3788
4026,262149738,01,051,030800,1,1047,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,L,N,21781,+32.5193761,-86.2155318,01051030800,010510308001,010510308001,NaN,3788
4027,262158441,01,051,030800,1,1069,None,36092,None,02100,01,051,030800,1,93456,None,None,None,None,None,None,None,None,None,None,None,None,None,None,01290,02,025,075,388,33860,None,None,None,None,L,N,11564,+32.5168904,-86.2165457,01051030800,010510308001,010510308001,NaN,3788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4129,210472546,01,051,030800,1,1070,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,L,N,30497,+32.5183078,-86.2151491,01051030800,010510308001,010510308001,NaN,3788
4130,210472550,01,051,030800,1,1056,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,L,N,75159,+32.5209906,-86.2082666,01051030800,010510308001,010510308001,NaN,3788
4131,210468676,01,051,030800,1,1056,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,025,031,388,33860,None,None,None,None,L,N,227,+32.5199621,-86.2103608,01051030800,010510308001,010510308001,NaN,3788
4132,210468684,01,051,030800,1,1043,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,None,None,None,None,None,None,None,None,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,2247,+32.5229752,-86.2086713,01051030800,010510308001,010510308001,NaN,3788


In [313]:
faces2 = faces1.merge(mini_geo[['GEOID', 'Logical Record Number']] , left_on = 'TGEOID', right_on = 'GEOID')
faces2.rename(columns = {'Logical Record Number':'TGEOID_short'}, inplace = True)
faces2

,TFID,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,BLOCKCE10,SUFFIX1CE,ZCTA5CE10,UACE10,PUMACE10,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,COUSUBFP,SUBMCDFP,ESTATEFP,CONCTYFP,PLACEFP,AIANNHFP,AIANNHCE,COMPTYP,TRSUBFP,TRSUBCE,ANRCFP,TTRACTCE,TBLKGPCE,ELSDLEA,SCSDLEA,UNSDLEA,CD116FP,SLDUST,SLDLST,CSAFP,CBSAFP,METDIVFP,CNECTAFP,NECTAFP,NCTADVFP,LWFLAG,OFFSET,ATOTAL,INTPTLAT,INTPTLON,GEOID_CT,GEOID_BG,GEOID_x,TGEOID,GEOID_short,GEOID_y,TGEOID_short
0,210466624,01,051,030800,1,1037,None,36092,94618,02100,01,051,030800,1,93456,None,None,None,81720,61430,2865,R,None,None,None,T00100,A,None,None,01290,02,030,031,388,33860,None,None,None,None,L,N,142774,+32.5284632,-86.2091482,01051030800,010510308001,010510308001,2865T00100A,3788,2865T00100A,3284
1,261054684,01,053,970500,1,1134,None,36502,None,02200,01,053,970500,1,90153,None,None,None,None,61430,2865,T,None,None,None,T00100,A,None,None,01350,01,022,066,426,12120,None,None,None,None,L,N,9559,+31.0716428,-87.4845597,01053970500,010539705001,010539705001,2865T00100A,3825,2865T00100A,3284
2,202585285,01,053,970500,1,1182,None,36502,None,02200,01,053,970500,1,90153,None,None,None,None,61430,2865,T,None,None,None,T00100,A,None,None,01350,01,022,066,426,12120,None,None,None,None,L,N,10058,+31.0714711,-87.4854106,01053970500,010539705001,010539705001,2865T00100A,3825,2865T00100A,3284
3,262740317,01,053,970500,1,1173,None,36502,None,02200,01,053,970500,1,90153,None,None,None,None,61430,2865,R,None,None,None,T00100,A,None,None,01350,01,022,066,426,12120,None,None,None,None,L,N,30656,+31.1026004,-87.4816080,01053970500,010539705001,010539705001,2865T00100A,3825,2865T00100A,3284
4,202586101,01,053,970500,1,1166,None,36502,None,02200,01,053,970500,1,90153,None,None,None,None,61430,2865,R,None,None,None,T00100,A,None,None,01350,01,022,066,426,12120,None,None,None,None,L,N,912,+31.1030612,-87.4832246,01053970500,010539705001,010539705001,2865T00100A,3825,2865T00100A,3284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64947,220494253,04,007,940200,1,1185,None,85941,None,00800,04,007,940200,1,91277,None,None,None,None,24320,1140,R,None,None,None,T00200,C,None,None,09160,01,007,007,429,37740,None,None,None,None,L,N,677,+33.8592196,-110.0548026,04007940200,040079402001,040079402001,1140T00200C,2538,1140T00200C,2867
64948,220489934,04,007,940200,1,1215,None,85941,None,00800,04,007,940200,1,91277,None,None,None,None,24320,1140,R,None,None,None,T00200,C,None,None,09160,01,007,007,429,37740,None,None,None,None,L,N,24280,+33.7923054,-110.0223485,04007940200,040079402001,040079402001,1140T00200C,2538,1140T00200C,2867
64949,220495925,04,007,940200,1,1224,None,85941,None,00800,04,007,940200,1,91277,None,None,None,10040,24320,1140,R,None,None,None,T00200,C,None,None,09160,01,007,007,429,37740,None,None,None,None,L,N,344,+33.7921575,-110.0267044,04007940200,040079402001,040079402001,1140T00200C,2538,1140T00200C,2867
64950,220494264,04,007,940200,1,1185,None,85941,None,00800,04,007,940200,1,91277,None,None,None,None,24320,1140,R,None,None,None,T00200,C,None,None,09160,01,007,007,429,37740,None,None,None,None,L,N,8774,+33.8726142,-110.0155739,04007940200,040079402001,040079402001,1140T00200C,2538,1140T00200C,2867


In [314]:
faces2.merge()

,AIANNHCE,TTRACTCE,TBLKGPCE,GEOID,NAMELSAD,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON
0,1185,T00100,A,1185T00100A,Tribal Block Group A,G2410,119741395,89526967,+42.9996415,-112.5831362
1,1185,T00100,B,1185T00100B,Tribal Block Group B,G2410,165166216,1452214,+43.0857723,-112.4519179
2,1185,T00100,C,1185T00100C,Tribal Block Group C,G2410,662404397,13888935,+42.7681940,-112.6483140
3,1185,T00100,D,1185T00100D,Tribal Block Group D,G2410,15525800,24887,+43.0246813,-112.4220207
4,1185,T00100,E,1185T00100E,Tribal Block Group E,G2410,477517189,2396677,+42.9753621,-112.1828263
...,...,...,...,...,...,...,...,...,...,...
912,1625,T00100,A,1625T00100A,Tribal Block Group A,G2410,478550853,1609050,+34.8829867,-106.5179484
913,1625,T00100,B,1625T00100B,Tribal Block Group B,G2410,372026076,440879,+34.9029292,-106.8728722
914,2680,T00100,A,2680T00100A,Tribal Block Group A,G2410,781606,0,+32.1307747,-111.0820491
915,2680,T00100,C,2680T00100C,Tribal Block Group C,G2410,161284,0,+32.1098493,-111.0781946


In [301]:
path = '/home/pio/zrp/zrp/data/processed/acs/2019/5yr/processed_Zest_ACS_Lookup_20195yr_blockgroup.parquet'
lookup_bg = pd.read_parquet(path)
lookup_bg

,GEOID,GEO_NAME,EXT_GEOID,B01003_001,B02001_001,B02001_002,B02001_003,B02001_004,B02001_005,B02001_006,...,B25075_023,B25075_024,B25075_025,B99021_001,B99021_002,B99021_003,B99162_003,B99162_004,B99162_005,B99162_007
0,010010201001,"Block Group 1, Census Tract 201, Autauga Count...",15000US010010201001,730,730,613,60,0,0,0,...,0,0,0,730,0,730,25,0,0,25
1,010010201002,"Block Group 2, Census Tract 201, Autauga Count...",15000US010010201002,1263,1263,1072,92,0,2,0,...,0,0,3,1263,8,1255,39,6,0,33
2,010010202001,"Block Group 1, Census Tract 202, Autauga Count...",15000US010010202001,835,835,276,507,0,0,21,...,0,0,0,835,12,823,14,0,0,14
3,010010202002,"Block Group 2, Census Tract 202, Autauga Count...",15000US010010202002,1124,1124,483,610,0,0,0,...,6,0,0,1124,25,1099,0,0,0,0
4,010010203001,"Block Group 1, Census Tract 203, Autauga Count...",15000US010010203001,2774,2774,1866,557,0,32,5,...,14,0,0,2774,256,2518,322,0,0,322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220328,721537506011,"Block Group 1, Census Tract 7506.01, Yauco Mun...",15000US721537506011,883,883,719,0,0,0,0,...,0,0,0,883,20,863,776,8,8,768
220329,721537506012,"Block Group 2, Census Tract 7506.01, Yauco Mun...",15000US721537506012,2523,2523,1946,89,51,0,0,...,11,0,0,2523,4,2519,2025,202,185,1823
220330,721537506013,"Block Group 3, Census Tract 7506.01, Yauco Mun...",15000US721537506013,991,991,538,227,0,0,0,...,0,0,0,991,8,983,839,10,0,829
220331,721537506021,"Block Group 1, Census Tract 7506.02, Yauco Mun...",15000US721537506021,1577,1577,1116,23,0,0,0,...,0,0,0,1577,131,1446,1433,246,246,1187


In [333]:
tribal_geo_list = list(mini_geo[mini_geo['Geography ID'].apply(lambda x: True if 'A' in x[-1] else False)]['Logical Record Number'])


In [335]:
raw_data = pd.read_csv('/home/pio/shared/zrp/shared_data/raw/acs/2019/5yr/Tracts_Block_Groups/e20195nm0080000.txt', sep=",", header=None)
raw_data[raw_data[5].apply(lambda x: x in tribal_geo_list)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187
1105,ACSSF,201900000.0,nm,0,80,1930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149,73,32,0,0,0,32,0,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,22,19,0,19,0,0,0,0,0,76,20,0,0,0,20,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56,13,43,0,0,0,0,0,0,0,0
1107,ACSSF,201900000.0,nm,0,80,1932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137,103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,17,0,35,0,0,35,51,0,35,16,34,34,0,0,0,9,9,0,0,25,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1198,ACSSF,201900000.0,nm,0,80,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460,280,74,57,47,10,13,0,8,5,4,4,0,0,0,0,0,0,31,0,0,0,0,4,27,0,15,8,7,48,0,22,26,112,7,92,13,180,90,40,40,0,0,0,0,0,32,27,0,5,0,18,11,7,16,6,0,0,0,10,0,0,58,13,45,5,0,5,0,11,0,11,0
1859,ACSSF,201900000.0,nm,0,80,2684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,224,105,12,4,4,0,8,0,8,0,0,0,0,0,0,0,0,0,40,0,0,0,0,8,32,0,6,6,0,7,0,0,7,40,6,12,22,119,83,24,24,0,0,0,0,0,38,0,0,38,0,21,21,0,30,0,0,0,0,30,0,0,6,0,6,0,0,0,0,0,0,0,0
1862,ACSSF,201900000.0,nm,0,80,2687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248,131,34,5,3,2,20,6,6,8,5,2,0,3,0,4,4,0,24,6,9,9,0,0,5,4,25,5,20,26,0,22,4,22,10,6,6,117,52,15,12,3,4,2,0,2,27,0,0,27,0,6,5,1,27,19,0,0,0,3,1,4,33,6,27,2,0,0,2,3,0,1,2
1864,ACSSF,201900000.0,nm,0,80,2689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [336]:
directory = '/home/pio/shared/zrp/shared_data/raw/acs/2019/5yr/2019_5yr_Summary_FileTemplates/'
for filename in os.listdir(directory):
    if "Geo" not in filename:
        break
        print(filename)

In [339]:
path = '/home/pio/shared/zrp/shared_data/raw/acs/2019/5yr/2019_5yr_Summary_FileTemplates/seq1.xlsx'

pd.read_excel(path).shape

(1, 241)

In [ ]:
path = '/home/pio/zrp/zrp/data/processed/geo/2019/Zest_Geo_Lookup_2019_State_01.parquet'

pd.read_parquet(path)

In [342]:
# Import edges geo files
import os
folder = f'/home/pio/shared/zrp/shared_data/raw/geo/2019/'
j = 0
data = []

for file_name in os.listdir(folder):
    if ('edges.shp' in file_name) & ('.xml' not in file_name) :
        file = folder + file_name
        
        dictList = []
        if indx == "address":
            with fiona.open(file) as src:
                for i in range(10000000):
                    dictList.append(next(src)["properties"])
        else:
            with fiona.open(file) as src:
                for i in range(len(src)):
                    dictList.append(next(src)["properties"])
        edge = pd.DataFrame(
            dictList,
            columns=dictList[0].keys(),
        )
        
        data.append(edge)
        j += 1
        if j == 100:
            break

edges = pd.concat(data)

### What proportion of block groups will need to be geocoded?

In [50]:
import sys
!{sys.executable} -m pip install fiona

Looking in indexes: https://pypi.org/simple, http://zamlpkgs-svc-nginx/artifactory/api/pypi/zest_pypi/simple/
     |████████████████████████████████| 62 kB 746 kB/s  eta 0:00:01
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
# Import raw geo files
import os
import pandas as pd
import fiona
folder = f'/home/pio/shared/zrp/shared_data/raw/geo/2019/'
j = 0
data = []
total_tbg = 0
total_bg = 0

for file_name in os.listdir(folder):
    if ('faces.shp' in file_name) & ('.xml' not in file_name) :
        file = folder + file_name
        
        dictList = []

        with fiona.open(file) as src:
            for i in range(len(src)):
                dictList.append(next(src)["properties"])

        face = pd.DataFrame(
            dictList,
            columns= ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'BLOCKCE10', 'AIANNHCE','TTRACTCE', 'TBLKGPCE'],
        )
        face  = face.drop_duplicates()
        
        face['is_tribal'] = ((face['AIANNHCE'].notna()) | (face['TTRACTCE'].notna()) | (face['TBLKGPCE'].notna())) * 1
        groups = face.groupby(['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE']).sum()['is_tribal']
        tbg = (groups > 0).sum()
        bg = (groups == 0).sum()
        
        total_tbg += tbg
        total_bg += bg

        j += 1
        print(j)


In [45]:
total_tbg

4940

In [48]:
total_bg

215800

In [47]:
total_tbg / total_bg

0.02289156626506024

### Checking speed of geocoder

In [8]:
import sys
!{sys.executable} -m pip install censusgeocode

Looking in indexes: https://pypi.org/simple, http://zamlpkgs-svc-nginx/artifactory/api/pypi/zest_pypi/simple/
  Using cached censusgeocode-0.5.2-py3-none-any.whl (9.2 kB)
  Using cached requests-2.28.0-py3-none-any.whl (62 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [46]:
import pandas as pd
import censusgeocode as cg
from random import uniform
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import time
import numpy as np

In [10]:
df_entry_data = pd.read_csv('/home/pio/shared/zrp/shared_data/processed/data/state_level/voters/CAL_cleaned.csv')

In [15]:
locations = df_entry_data[['voter_id', 'res_address_1','res_state', 'res_city', 'res_zip_code' ]][0:1000]

def geocode(row):
    index, voter_id, adr, state, city, zip_code = row
    try:
        census = cg.address(adr, city=city, state=state, zip=zip_code)[0]['geographies']['2020 Census Blocks'][0]

        data = dict(block = census['BLOCK'], 
                    index = index)

    except Exception as e:
        data = dict(index = index)

    return data
start = time.time()
for i in range(50):    
    with ThreadPoolExecutor() as tpe:
         data = list(tqdm(tpe.map(geocode, locations.itertuples()), total=len(locations)))
    print(i)
end = time.time()
df = pd.DataFrame.from_records(data)


0



1



2



3



4


...


45



46



47



48



49


In [16]:
((end-start)/len(locations) /50) * 1000000 * 0.023 /60

5.078228240728378

In [ ]:
result = cg.onelineaddress('15872  SW SR 73, FL, CLARKSVILLE, 32430')
result

In [180]:
cg = censusgeocode.CensusGeocode(benchmark='Public_AR_Census2021',vintage='Census2010_Census2021')

In [ ]:
import censusgeocode



start = time.time()
results = []
for n in range(100):   
    result = cg.address(data['res_address_1'][n], 
                        city = data['res_city'][n],
                        state = data['res_state'][n],
                        zip = str(data['res_zip_code'][n]))
    result
    break
    
    if len(result) != 0:
        results.append(result[0]["geographies"]['2020 Census Blocks'][0]['BLOCK'])
end = time.time()

In [ ]:
end-start

In [88]:
result = cg.coordinates(x=-76, y=41)

In [77]:
n = 2
loc = geocoder.location.from_address(street_1 = data['res_address_1'][n],
                                          city = data['res_city'][n],
                                          state = data['res_state'][n],
                                          zip_code = str(data['res_zip_code'][n]))

In [ ]:
print(loc.matched_addresses[0].address)

### Assesment of features and feature selection

In [31]:
df_entry_data

,voter_id,last_name,first_name,middle_name,name_suffix,res_address_1,res_address_2,res_city,res_state,res_zip_code,reg_date,birthdate,gender,race
0,100640898,SMALL,HAROLD,WADE,NaN,20777 CENTRAL AVE E,APT 6,BLOUNTSTOWN,FL,32424,08/10/1992,11/15/1951,M,BLACK
1,100660138,THOMAS,LARRY,EUGENE,NaN,2263 NW SMOKEY LN,NaN,FOUNTAIN,FL,32438,06/11/1996,05/10/1954,M,WHITE
2,100653579,PHILLIPS,JEAN,ANN,NaN,15872 SW SR 73,NaN,CLARKSVILLE,FL,32430,07/28/1995,08/20/1962,F,WHITE
3,100694125,AMMONS,SHERRI,LENA,NaN,7633 SW CR 392,NaN,YOUNGSTOWN,FL,32466,02/08/2001,07/09/1971,F,WHITE
4,100690603,MCNEAL,SANDRA,DENISE,NaN,6471 SW SR 73,NaN,KINARD,FL,32449,09/20/2000,03/11/1980,F,WHITE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8441,128760624,MCGILL,MARK,EVERETT,NaN,17460 NW GUILFORD CIR,NaN,BLOUNTSTOWN,FL,32424,09/19/2020,11/06/1998,M,WHITE
8442,128795950,STORY,NANCY,LEE,NaN,18899 NE SR 69,NaN,BLOUNTSTOWN,FL,32424,09/22/2020,01/16/1951,M,WHITE
8443,126118392,JOHNSON,JACOB,ALAN,NaN,21769 NE W L GODWIN RD,NaN,BLOUNTSTOWN,FL,32424,09/28/2020,12/29/1998,M,WHITE
8444,128901910,JONES,DALLAS,LYNN,NaN,27178 SR 71 NORTH,NaN,ALTHA,FL,32421,09/30/2020,09/26/2002,M,AIAN


In [63]:
num_of_chunks = int(len(df_entry_data)/1000) + 1

def geocode(row):
    index, voter_id, adr, state, city, zip_code = row
    try:
        census = cg.address(adr, city=city, state=state, zip=zip_code)[0]['geographies']['2020 Census Blocks'][0]

        data = dict(block = census['BLOCK'],
                    block_grp = census['BLKGRP'],
                    tract = census['TRACT'],
                    county = census['COUNTY'],
                    state = census['STATE'],
                    geoid = census['GEOID'],
                    voter_id = voter_id)

    except Exception as e:
        data = dict(index = index)

    return data
start = time.time()

for i in range(num_of_chunks):
    locations = df_entry_data[['voter_id', 'res_address_1','res_state', 'res_city', 'res_zip_code' ]][i*1000:(i+1)*1000]
    with ThreadPoolExecutor() as tpe:
        data = list(tqdm(tpe.map(geocode, locations.itertuples()), total=len(locations)))
    if i == 0:
        df = pd.DataFrame.from_records(data)
    else:
        df_chunk = pd.DataFrame.from_records(data)
        df = df.append(df_chunk)

end = time.time()

In [64]:
(end-start)/len(df_entry_data)*1000000*0.023/60

5.166636641790162

In [29]:
# n=2
# cg.address(df_entry_data['res_address_1'][n], city =  df_entry_data['res_city'][n], state = df_entry_data['res_state'][n], zip = df_entry_data['res_zip_code'][n])

In [65]:
df_scored = df.merge(df_entry_data[['voter_id', 'race']], on = 'voter_id', how = 'left')
df_scored["is_AIAN"] = (df_scored['race'] == 'AIAN')

In [66]:
df_scored['is_AIAN'].sum()

74

In [67]:
df_scored

,block,block_grp,tract,county,state,geoid,voter_id,index,race,is_AIAN
0,1002,1,010302,013,12,120130103021002,100640898.0,NaN,BLACK,False
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,False
2,1058,1,010200,013,12,120130102001058,100653579.0,NaN,WHITE,False
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,False
4,2045,2,010200,013,12,120130102002045,100690603.0,NaN,WHITE,False
...,...,...,...,...,...,...,...,...,...,...
8441,2017,2,010301,013,12,120130103012017,128760624.0,NaN,WHITE,False
8442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8442.0,NaN,False
8443,1051,1,010100,013,12,120130101001051,126118392.0,NaN,WHITE,False
8444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8444.0,NaN,False
